In [ ]:
# default_exp dlc_importer

# DLCImporter

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *
from fastcore.imports import operator

In [ ]:
#export
import pandas as pd

class DLCImporter:
    """Used to import DLC result files."""
    
    def import_hdf(self, file):
        df = pd.read_hdf(file)
        df.columns = df.columns.droplevel(0) # drop redundant scorer
        return df

`DLCImporter` can be used to import result files from DeepLabCut in H5 format.
It will automatically drop the unnecessary `scorer` level.

In [ ]:
imp = DLCImporter()
df = imp.import_hdf('example_data/coordinates.h5')
display(df)

bodyparts        head                               beak              \
coords              x           y likelihood           x           y   
0          773.376465  231.518768   0.999999  726.495178  235.638046   
1          773.129822  231.487213   0.999999  725.662231  235.242844   
2          773.009827  231.793518   0.999999  726.025696  235.272522   
3          773.748779  231.791260   0.999999  726.288940  235.864319   
4          774.934326  231.623734   0.999999  726.298279  235.749908   
..                ...         ...        ...         ...         ...   
95         691.788513  232.490265   1.000000  673.796082  238.801743   
96         691.545410  232.707413   1.000000  673.634888  238.658234   
97         691.117371  232.242767   1.000000  673.748840  239.055954   
98         691.294067  232.225220   1.000000  673.927002  239.141891   
99         691.483643  232.269226   1.000000  673.797241  239.390625   

bodyparts              left_neck                         right_neck  ...  \
coords    likelihood           x           y likelihood           x  ...   
0           0.999981  726.502014  277.634125   0.999998  803.271179  ...   
1           0.999951  725.964478  278.003082   0.999999  803.197144  ...   
2           0.999978  725.764893  278.884918   0.999998  802.567810  ...   
3           0.999985  725.889465  279.045715   0.999998  803.356934  ...   
4           0.999990  726.302551  278.388367   0.999999  802.530273  ...   
..               ...         ...         ...        ...         ...  ...   
95          0.018886  697.399841  282.134796   0.999998  737.725342  ...   
96          0.016135  697.256165  283.058899   0.999999  736.505920  ...   
97          0.007289  696.269043  282.351929   0.999999  735.976685  ...   
98          0.004682  695.629456  282.407013   1.000000  735.639404  ...   
99          0.010126  695.367371  281.720947   0.999999  735.199585  ...   

bodyparts         b1          b2                                 b3  \
coords    likelihood           x           y likelihood           x   
0           0.999986  731.942932  212.751831   0.999972  637.101440   
1           0.999984  732.492004  212.608414   0.999969  636.351440   
2           0.999989  732.367859  212.449570   0.999967  636.567993   
3           0.999988  732.378052  212.362946   0.999962  636.978821   
4           0.999991  732.364014  212.297073   0.999969  637.059021   
..               ...         ...         ...        ...         ...   
95          0.999981  729.320374  214.642715   0.999988  636.506165   
96          0.999983  729.624817  215.617279   0.999986  636.087585   
97          0.999982  729.727722  214.848831   0.999987  636.203613   
98          0.999983  729.409668  214.924347   0.999989  636.108765   
99          0.999981  729.390991  215.064926   0.999983  636.145447   

bodyparts                                 b4                         
coords              y likelihood           x           y likelihood  
0          275.546967   0.999044  720.761475  260.645416   0.004323  
1          274.908783   0.999068  720.861084  260.821045   0.003748  
2          275.915558   0.999465  721.077087  261.231415   0.003017  
3          276.222534   0.999626  721.093933  261.198242   0.004915  
4          276.169647   0.999717  720.964783  260.974213   0.003497  
..                ...        ...         ...         ...        ...  
95         275.310120   0.999486  742.268372  243.916214   0.001441  
96         275.022858   0.999226  741.823853  243.833771   0.001208  
97         275.289307   0.999269  741.812866  243.337326   0.000828  
98         274.952759   0.999238  987.140259  601.538696   0.000869  
99         274.386780   0.999225  742.091919  245.363281   0.000920  

[100 rows x 60 columns]

In [ ]:
#export
import matplotlib
import matplotlib.pyplot as plt

plt.rc('animation', html='jshtml')

class PigeonAnimator:
    """Animates a pigeon skeleton using matplotlib"""
    
    def __init__(self, skeleton, figsize=(10,6), xlim=(0, 1000), ylim=(0, 1000)):
        self.skeleton,self.figsize,self.xlim,self.ylim = skeleton,figsize,xlim,ylim
        
    def visualize(self, df, start, end):
        fig = plt.figure(figsize=self.figsize)
        ax = plt.axes(xlim=self.xlim, ylim=self.ylim)
        anim_bones = [ax.plot([], [], lw=2)[0] for _ in self.skeleton]
        
        def animate(i):
            data = df.iloc[[i]]
            
            for s, b in zip(self.skeleton, anim_bones):
                b.set_data([data[s[0],'x'], data[s[1],'x']], [data[s[0],'y'], data[s[1],'y']])
        
        plt.close()
    
        anim = matplotlib.animation.FuncAnimation(fig, animate, frames=range(start, end))
        return anim

You can animate a set of frames using the `PigeonAnimator`. For this, you construct the animator by providing a list of tuple representing the skeleton.

In [ ]:
p_anim = PigeonAnimator([('body', 'head'), ('head', 'left_neck'), ('head', 'right_neck')])
p_anim.visualize(df, 0, 60)

In [ ]:
#export
class PigeonAnimatorFactory:
    DEFAULT = PigeonAnimator([('left_neck', 'right_neck'), ('head', 'left_neck'), ('head', 'right_neck'), ('left_neck', 'tail'), ('right_neck', 'tail'),
                             ('left_neck', 'left_up_wing'), ('left_up_wing', 'left_middle_wing'), ('left_middle_wing', 'left_down_wing'), ('left_up_wing', 'left_down_wing'),
                             ('right_neck', 'right_up_wing'), ('right_up_wing', 'right_middle_wing'), ('right_middle_wing', 'right_down_wing'), ('right_up_wing', 'right_down_wing')])

There are also different pre-configured animators at your disposal.

In [ ]:
PigeonAnimatorFactory.DEFAULT.visualize(df, 0, 60)